###   本模块在 DAI-Net 项目中扮演了图像增强的核心角色，旨在提升低光照图像的质量，为目标检测任务提供更清晰的输入数据。包含了 DecomNet、RetinexNet、ZeroDCE 和 Enhancer 等模块，通过深度学习和 Retinex 理论实现图像分解与增强，具体作用如下：它首先通过 ZeroDCE或 RetinexNet将图像分解为反射率和光照分量，随后，Enhancer 类根据参数选择合适的增强方法，确保灵活性和兼容性；最终，增强后的图像被输入到目标检测模型中，显著改善夜间或低光照场景下的检测性能，从而提升整个系统在日夜转换场景下的鲁棒性和准确性。

## 1.导入所需模块

In [ ]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F

### 2.原项目中基于 PyTorch 的神经网络模块，通过卷积层和 ReLU 激活函数实现低光照图像的分解，将输入图像分解为反射率（R）和光照（L）两个部分，用于图像增强任务。

In [24]:
class DecomNet(nn.Module):
    def __init__(self, channel=64, kernel_size=3):
        super(DecomNet, self).__init__()
        # Shallow feature extraction
        self.net1_conv0 = nn.Conv2d(4, channel, kernel_size * 3,
                                    padding=4, padding_mode='replicate')
        # Activated layers!
        self.net1_convs = nn.Sequential(nn.Conv2d(channel, channel, kernel_size,
                                                  padding=1, padding_mode='replicate'),
                                        nn.ReLU(),
                                        nn.Conv2d(channel, channel, kernel_size,
                                                  padding=1, padding_mode='replicate'),
                                        nn.ReLU(),
                                        nn.Conv2d(channel, channel, kernel_size,
                                                  padding=1, padding_mode='replicate'),
                                        nn.ReLU(),
                                        nn.Conv2d(channel, channel, kernel_size,
                                                  padding=1, padding_mode='replicate'),
                                        nn.ReLU(),
                                        nn.Conv2d(channel, channel, kernel_size,
                                                  padding=1, padding_mode='replicate'),
                                        nn.ReLU())
        # Final recon layer
        self.net1_recon = nn.Conv2d(channel, 4, kernel_size,
                                    padding=1, padding_mode='replicate')

    def forward(self, input_im):
        input_max = torch.max(input_im, dim=1, keepdim=True)[0]
        input_img = torch.cat((input_max, input_im), dim=1)
        feats0 = self.net1_conv0(input_img)
        featss = self.net1_convs(feats0)
        outs = self.net1_recon(featss)
        R = torch.sigmoid(outs[:, 0:3, :, :])
        L = torch.sigmoid(outs[:, 3:4, :, :])
        return R, L

### 3.基于 PyTorch 的模块，旨在为图像增强任务提供灵活的选择，通过参数决定使用 ZeroDCE 或 RetinexNet 进行处理，并在 forward 方法中根据选择调用相应的增强器，返回增强后的图像及可能的附加输出，以适应不同场景的增强需求。

In [2]:
# This Retinex Decom Net is frozen during training of DAI-Net
class Enhancer(nn.Module):
    def __init__(self, use_zerodce=True):
        super().__init__()
        self.use_zerodce = use_zerodce
        if use_zerodce:
            self.enhancer = ZeroDCE()
        else:
            self.enhancer = RetinexNet()
        
    def forward(self, x):
        if self.use_zerodce:
            enhanced, _ = self.enhancer(x)
            return enhanced, None  # 保持接口兼容
        else:
            return self.enhancer(x)

NameError: name 'nn' is not defined

In [26]:
class RetinexNet(nn.Module):
    def __init__(self):
        super(RetinexNet, self).__init__()
        self.DecomNet = DecomNet()

    def forward(self, input):
        R, I = self.DecomNet(input)
        return R, I

# ZeroDCE模块

#### 基于 PyTorch 的轻量级图像增强模块，通过深度学习技术提升低光照图像的质量。其核心方法是将输入图像分解为反射率（R）和光照（I）两个部分，并通过调整光照生成增强图像，适用于夜间目标检测等场景。模块包含以下关键函数：

## __init__ 函数
#### 该函数初始化 ZeroDCE 模型，定义了一个包含 8 层卷积的网络，通道数从 3（RGB 输入）逐步增加到 128，再减少到 4（3 通道反射率 R 和 1 通道光照 I），并设置 ReLU 激活函数以提取非线性特征。

## forward 函数
#### forward 函数执行前向传播，将输入图像通过卷积层处理，分解为反射率 R 和光照 I（使用 sigmoid 激活限制范围在 [0, 1]），随后调整光照 I（例如乘以 1.5 并裁剪），最后通过 R 和调整后的 I 相乘生成增强图像，返回增强图像、R 和 I。

## loss 函数
#### loss 函数定义了训练时的损失计算，包括重建损失（确保 R 和 I 重构原始图像）、互重建损失（增强 R 和 I 的独立性）、反射率一致性损失（强制低光和高光图像的 R 相似）以及光照平滑损失（确保 I 平滑），通过加权组合优化分解和增强质量。

## smooth 函数
#### smooth 函数实现光照平滑损失，首先将反射率 R 转换为灰度图，然后计算光照 I 在 x 和 y 方向上的加权梯度（加权基于 R 的灰度梯度），最后返回梯度的平均值，确保光照 I 在非边缘区域平滑，同时在边缘处保留细节。

In [27]:
class ZeroDCE(nn.Module):
    def __init__(self):
        super().__init__()
        self.relu = nn.ReLU(inplace=True)
        
        # 定义卷积层：逐步增加和减少通道数
        self.conv1 = nn.Conv2d(3, 32, 3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 64, 3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(64, 32, 3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(32, 16, 3, stride=1, padding=1)
        self.conv7 = nn.Conv2d(16, 8, 3, stride=1, padding=1)
        
        # 输出层：输出 4 通道（R 3 通道 + I 1 通道）
        self.conv8 = nn.Conv2d(8, 4, 3, stride=1, padding=1)

    def forward(self, x):
        # 前向传播
        x1 = self.relu(self.conv1(x))
        x2 = self.relu(self.conv2(x1))
        x3 = self.relu(self.conv3(x2))
        x4 = self.relu(self.conv4(x3))
        x5 = self.relu(self.conv5(x4))
        x6 = self.relu(self.conv6(x5))
        x7 = self.relu(self.conv7(x6))
        
        # 分解输出：4 通道分为 R（前 3 通道）和 I（第 4 通道）
        decomposition = self.conv8(x7)
        R = torch.sigmoid(decomposition[:, :3, :, :])  # 反射率 R，3 通道，范围 [0, 1]
        I = torch.sigmoid(decomposition[:, 3:4, :, :])  # 光照 I，单通道，范围 [0, 1]
        
        # 调整光照 I（示例：提升亮度）
        adjusted_I = I * 1.5  # 可根据需求调整倍数
        adjusted_I = torch.clamp(adjusted_I, 0, 1)  # 限制在 [0, 1] 范围内
        
        # 生成增强图像
        enhanced = R * adjusted_I
        return enhanced, R, I

    def loss(self, R_low, I_low, R_high, I_high, input_low, input_high):
        # 重建损失：确保 R * I 重建输入图像
        recon_loss_low = F.l1_loss(R_low * I_low, input_low)
        recon_loss_high = F.l1_loss(R_high * I_high, input_high)
        
        # 互重建损失：增强跨域一致性
        recon_loss_mutal_low = F.l1_loss(R_high * I_low, input_low)
        recon_loss_mutal_high = F.l1_loss(R_low * I_high, input_high)
        
        # 反射率一致性损失：低光和高光图像的 R 应相似
        equal_R_loss = F.l1_loss(R_low, R_high.detach())
        
        # 光照平滑损失：确保 I 在空间上平滑
        Ismooth_loss_low = self.smooth(I_low, R_low)
        Ismooth_loss_high = self.smooth(I_high, R_high)

        # 总损失：加权组合
        loss = (recon_loss_low +
                recon_loss_high +
                0.001 * recon_loss_mutal_low +
                0.001 * recon_loss_mutal_high +
                0.1 * Ismooth_loss_low +
                0.1 * Ismooth_loss_high +
                0.01 * equal_R_loss)
        return loss

    def gradient(self, input_tensor, direction):
        """计算梯度，用于光照平滑损失"""
        smooth_kernel_x = torch.FloatTensor([[0, 0], [-1, 1]]).view((1, 1, 2, 2)).to(input_tensor.device)
        smooth_kernel_y = torch.transpose(smooth_kernel_x, 2, 3)
        kernel = smooth_kernel_x if direction == "x" else smooth_kernel_y
        grad_out = torch.abs(F.conv2d(input_tensor, kernel, stride=1, padding=1))
        return grad_out

    def ave_gradient(self, input_tensor, direction):
        """计算平均梯度"""
        return F.avg_pool2d(self.gradient(input_tensor, direction), kernel_size=3, stride=1, padding=1)

    def smooth(self, input_I, input_R):
        """光照平滑损失函数"""
        # 将 R 转换为灰度图，用于指导光照平滑
        input_R_gray = 0.299 * input_R[:, 0, :, :] + 0.587 * input_R[:, 1, :, :] + 0.114 * input_R[:, 2, :, :]
        input_R_gray = torch.unsqueeze(input_R_gray, dim=1)
        
        # 计算光照的梯度并根据 R 加权
        grad_x = self.gradient(input_I, "x") * torch.exp(-10 * self.ave_gradient(input_R_gray, "x"))
        grad_y = self.gradient(input_I, "y") * torch.exp(-10 * self.ave_gradient(input_R_gray, "y"))
        return torch.mean(grad_x + grad_y)